# SDG 15.3.1 Error Recode Script Example

This notebook demonstrates how to use the SDG 15.3.1 error recode script via the trends.earth API. The script supports both single-period and multi-period land degradation jobs and allows for recoding specific areas based on error polygons.

## 1. Setup Environment and Import Libraries

Before running this notebook, make sure you have:

1. **Installed dependencies**: `pip install -r requirements.txt`
2. **Created environment file**: Copy `.env.example` to `.env` and update with your credentials:
   ```bash
   cp .env.example .env
   ```
3. **Updated credentials** in the `.env` file:
   ```
   API_BASE_URL=https://api.trends.earth
   API_USERNAME=your_actual_username
   API_PASSWORD=your_actual_password
   ```

In [7]:
import json
import uuid
import warnings
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Import the shared API client
import folium
import rasterio
import rasterio.mask
from trendsearth_api import TrendsEarthAPIClient

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

print("Libraries imported successfully!")
print("Environment variables loaded from .env file")

Libraries imported successfully!
Environment variables loaded from .env file


## 2. Configure API Connection and Authentication

In [8]:
# Initialize the API client
api_client = TrendsEarthAPIClient()

# Authenticate using environment variables from .env file
if api_client.authenticate_from_env():
    print("Ready to submit error recode jobs!")
    session = api_client.session  # For compatibility with existing code
else:
    print("Authentication failed - check your .env file")
    print("Please ensure your .env file contains:")
    print("- API_USERNAME=your_username")
    print("- API_PASSWORD=your_password")
    print("- API_BASE_URL=https://api.trends.earth (optional)")
    session = None

Using API URL from environment: https://api.trends.earth
Authenticating with Trends.Earth API...
   Using email: trends.earth-prais-server@trends.earth
Successfully authenticated with Trends.Earth API
Ready to submit error recode jobs!


## 3. Define Example Parameters for Error Recoding

### Multi-period Data Support

The  error recode script uses the `filters` parameter to select specific reporting periods for multi-period land degradation jobs:

**For Single-period Jobs:**
```python
"filters": []  # No filters needed
```

**For Multi-period Jobs:**
```python
"filters": [{"field": "reporting_year_final", "value": 2020}]  # Target specific year
```

### Error Polygon Schema Requirements

The error polygons must follow the trends.earth schema validation requirements:

**Required Properties:**
- `uuid`: A unique identifier (auto-generated UUID string)
- `type`: Must be "Feature" for each feature

**Optional Properties:**
- `location_name`: Descriptive name for the error area
- `area_km_sq`: Area in square kilometers
- `process_driving_change`: Description of what caused the error
- `basis_for_judgement`: Justification for the correction

**Recode Options** (all optional, use None to leave unchanged):
- `recode_deg_to`: What to recode degraded pixels to
  - `None`: No change (default)
  - `-32768`: No data
  - `0`: Stable
  - `1`: Improved
- `recode_stable_to`: What to recode stable pixels to
  - `None`: No change (default) 
  - `-32768`: No data
  - `-1`: Degraded
  - `1`: Improved
- `recode_imp_to`: What to recode improved pixels to
  - `None`: No change (default)
  - `-32768`: No data
  - `-1`: Degraded
  - `0`: Stable

In [9]:
with open("ATG_polygon.geojson", "r") as f:
    atg_polygon = json.load(f)

params = {
    "iso": "ATG",
    "boundary_dataset": "UN",
    "write_tifs": False,
    "aoi": json.dumps(atg_polygon),
    "error_polygons": {
        "type": "FeatureCollection",
        "name": "test error recode",
        "crs": {
            "type": "name",
            "properties": {"name": "urn:ogc:def:crs:OGC:1.3:CRS84"},
        },
        "features": [
            {
                "type": "Feature",
                "properties": {
                    "uuid": str(uuid.uuid4()),
                    "location_name": "False degradation - mining misclassified",
                    "area_km_sq": 0.04,
                    "process_driving_change": "Open pit mining misclassified as degradation",
                    "basis_for_judgement": "Ground truth verification shows managed mining operation",
                    "recode_deg_to": 0,  # Recode degraded pixels to stable
                    "recode_stable_to": None,  # No change to stable pixels
                    "recode_imp_to": None,  # No change to improved pixels
                    "periods_affected": ["baseline"],
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [-61.85, 17.05],
                            [-61.8, 17.05],
                            [-61.8, 17.1],
                            [-61.85, 17.1],
                            [-61.85, 17.05],
                        ]
                    ],
                },
            },
            {
                "type": "Feature",
                "properties": {
                    "uuid": str(uuid.uuid4()),
                    "location_name": "Restoration success not captured",
                    "area_km_sq": 0.02,
                    "process_driving_change": "Successful restoration not detected by algorithm",
                    "basis_for_judgement": "Field monitoring shows vegetation recovery",
                    "recode_deg_to": 1,  # Recode degraded pixels to improved
                    "recode_stable_to": 1,  # Also recode stable pixels to improved
                    "recode_imp_to": None,  # Keep improved pixels as is
                    "periods_affected": ["baseline", "reporting_2"],
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [-61.82, 17.08],
                            [-61.78, 17.08],
                            [-61.78, 17.12],
                            [-61.82, 17.12],
                            [-61.82, 17.08],
                        ]
                    ],
                },
            },
            {
                "type": "Feature",
                "properties": {
                    "uuid": str(uuid.uuid4()),
                    "location_name": "Data quality issue",
                    "area_km_sq": 0.01,
                    "process_driving_change": "Cloud contamination in satellite data",
                    "basis_for_judgement": "Visual inspection shows persistent cloud cover",
                    "recode_deg_to": -32768,  # Set all to no data due to poor data quality
                    "recode_stable_to": -32768,  # Set all to no data
                    "recode_imp_to": -32768,  # Set all to no data
                    "periods_affected": ["reporting_2"],
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [-61.88, 17.15],
                            [-61.84, 17.15],
                            [-61.84, 17.18],
                            [-61.88, 17.18],
                            [-61.88, 17.15],
                        ]
                    ],
                },
            },
        ],
    },
}

## 4. Submit Job Execution to API

In [10]:
if api_client.access_token:
    print(" Submitting error recode job...")
    execution_id = api_client.submit_job("sdg-15-3-1-error-recode-2-1-17", params)

    if execution_id:
        print("Job submitted. Monitoring job status...")

        # Monitor the job until completion
        final_status = api_client.monitor_job(execution_id, max_minutes=15)

        if final_status:
            status = final_status.get("status", "unknown").upper()
            if status in ["SUCCESS", "FINISHED"]:
                print("✅ Error recode job completed successfully!")

                # The results should be available in the job data
                results = final_status.get("results", {})
                if results:
                    print("Results available")
                    pprint(results)
                else:
                    print("No results data found in job response")
        else:
            print("❌ Failed to monitor job")
    else:
        print("❌ Failed to submit job")
else:
    print("❌ Cannot submit job: not authenticated")
    print("Please check your credentials and authentication")

 Submitting error recode job...
Job submitted. Monitoring job status...
Status: READY
Status: FAILED
Job failed!
